# Imports and File Uploading


In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files, drive

### Mount Drive and Read a File

In [ ]:
drive.mount('/content/drive')

In [ ]:
root = '/content/drive/My Drive/STEM Fellowship - Undergrad BDC 2020/FolderName/'
df = pd.read_csv(root + 'filename.csv')

### Mount Drive and Read all Files in a Folder Saving Them to their Original File Name in a List of Data Frames

In [ ]:
drive.mount('/content/drive')
!ls /content/drive/

csvs = !ls /content/drive/My\ Drive/STEM\ Fellowship\ -\ Undergrad\ BDC\ 2020/Datasets/US\ Male\ Unemployment\ CSV\ \(for\ #7\)/

split = []
for item in csvs:
  split.extend(item.split())

split

df = []

root = '/content/drive/My Drive/STEM Fellowship - Undergrad BDC 2020/Datasets/US Male Unemployment CSV (for #7)/'
for i in range(len(split)):
  df.append(pd.read_csv(root + f'{split[i]}'))
  df[i].name = f'{split[i]}'

print(df[0].name, df)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'My Drive'
16-19.csv [   Labor Force Statistics from the Current Population Survey  ... Unnamed: 13
0                                 Original Data Value         ...         NaN
1                                                 NaN         ...         NaN
2                                          Series Id:         ...         NaN
3                             Not Seasonally Adjusted         ...         NaN
4                                       Series title:         ...         NaN
..                                                ...         ...         ...
79                                               2016         ...         925
80                                               2017         ...         827
81                                               2018         ...         759
82                                               2019         ...    

### Load Files from Computer and Merge all Into a Single Data Frame Only Keeping One Column of Interest for Each Column

In [ ]:
uploaded = files.upload()

In [ ]:
# all the data sets for both economic indicators and sectors have the headings:
# Date, Open, High, Low, Close, Adj Close, Volume

# column to keep from the data and do analysis on:
Col_of_interest = 'Adj Close'

# create some large range of dates (daily)
time = np.arange('1920-01', '2020-06', dtype='datetime64[D]')
# create a master data frame with this date data and name its column 'Date'
master = pd.DataFrame(time)
master.columns = ['Date']

#this section for plotting histotical time series NOT shared by all the data sets:
test = pd.DataFrame(time)
test.columns = ['Date']
test = test.set_index('Date')
#end of this section^

# for every file that was uploaded read it as a data frame, keep only the columns of interest, and merge it to the master
for thing in uploaded:
  temp = pd.read_csv(thing)
  temp = temp[['Date',f'{Col_of_interest}']]
  temp.Date = pd.to_datetime(temp.Date, format='%d-%m-%Y')
  temp.name = f'{thing}'

  #this section is for plotting historical time series NOT shared by all the data sets:
  temp2 = temp
  temp2 = temp2.set_index('Date')
  test = pd.merge(test, temp2, left_index=True, right_index=True, how='outer')
  test.columns = [*test.columns[:-1], f'{str(temp.name)[:-4]}']
  #end of this section^

  master = pd.merge(left = master, right = temp, left_on='Date', right_on='Date') # merge on the date column, any missing date is removed
  master.columns = [*master.columns[:-1], f'{str(temp.name)[:-4]}'] # rename the last column as the original file name but remove the '.csv' from the end

test['Date'] = test.index
test.index = np.arange(0,test.shape[0])
col = test.pop("Date") # cut the Months column from where it was at the end of the table
test.insert(0, col.name, col) # insert the column at column 0 of the data frame

# check that things worked
test

# To Single Column Matrix
A lot of the unemployment data is given in a grid with year on the vertical axis and months as the column headers but I want it as a single column for easier plotting. 

In [ ]:
# create a single column dataframe from a grid of data for (year x months)

def matrix_to_column(df):
  time_axis = []
  one_col = []

  # create a vector for the date values available in the dataframe
  for i in range(int(df.iat[0,0]), int(df.iat[-1,0])+1):
    for j in range(1,13):
      time_axis.append(f'{i}-{j}-01')

  # move the data into a single column
  for i in range(0,df.shape[0]):
    for j in range(1,13):
        one_col.append(df.iat[i,j])

  output = pd.DataFrame(data = one_col, index = time_axis) 
  output.columns = ['Percent']
  output['Date'] = output.index
  col = output.pop("Date") # cut the date column from where it was at the end of the table
  output.insert(0, col.name, col) # insert the column at column 0 of the data frame
  output.index = np.arange(0,output.shape[0])

  temp = output.Percent.last_valid_index()
  output = output.iloc[:temp+1,:]
  output.Date = pd.to_datetime(output.Date)
  output.Percent = output.Percent.astype('float')

  return output

CPI = matrix_to_column(cpi)
CPI

# Create a String of Months

In [ ]:
time1 = np.arange('1900-01', '2021-01', dtype='datetime64[D]')

# Plot a Single Line Within a Date Range

In [ ]:
start_date1 = test.loc[test.Date == np.datetime64('1955-03-14')].index[0]
end_date1 = test.loc[test.Date == np.datetime64('1960-03-14')].index[0]

fig,ax = plt.subplots()
plt.xticks(rotation = 'vertical')
fig.set_size_inches(16,9)
fig.set_facecolor('w')

plt.plot(test.iloc[start_date1:end_date1,0], test.iloc[start_date1:end_date1, 5])

plt.legend()
plt.title("Adj Closed Price of S&P500 During 1957 Asian Flu")
plt.xlabel("Date")
plt.show()

# Plot Multiple Columns in a Data Frame

In [ ]:
start_date= master.loc[master.Date == np.datetime64('2019-11-14')].index[0]
end_date= master.loc[master.Date == np.datetime64('2019-11-14')].index[0]

fig,ax = plt.subplots()
plt.xticks(rotation = 'vertical')
fig.set_size_inches(16,9)
fig.set_facecolor('w')

for column in range(1,master.shape[1]):
  plt.plot(master.iloc[start_date:,0], master.iloc[start_date:, column],label = f'{master.iloc[:,column].name}')

plt.legend()
plt.title("Adj Closed Price of Indicators")
plt.xlabel("Date")
plt.show()


# Coloured Markdown Text


In [ ]:
<font color='red'>bar</font>

# Create Column for a Simply Moving Average of Another Columns

In [ ]:
dji['High_SMA'] = dji.loc[:,['High']].rolling(window=14).mean()